# Identify a Case for the AMS Open Radar Short Course 2023

## Imports

In [ ]:
import glob
import fsspec
# import s3f3
import pyart
import matplotlib.pyplot as plt
import pydda
# import warnings
# warnings.filterwarnings("ignore")

In [ ]:
!pip3 install fsspec

## April 2008

### UAH ARMOR Radar

In [ ]:
files = glob.glob('../../data/uah-armor/*')
radar = pyart.io.read(files[1])

In [ ]:
nyquist = radar.instrument_parameters['nyquist_velocity']['data'][0]
vel_dealias = pyart.correct.dealias_region_based(radar,
                                                    vel_field='velocity',
                                                    nyquist_vel=nyquist,
                                                    centered=True,
                                                    )
radar.add_field('corrected_velocity', vel_dealias, replace_existing=True)

In [ ]:
display = pyart.graph.RadarDisplay(radar)
display.plot('reflectivity',
             vmin=-20,
             vmax=70,
             cmap='pyart_NWSRef',
             sweep=0)

In [ ]:
display = pyart.graph.RadarDisplay(radar)
display.plot('corrected_velocity',
             vmin=-30,
             vmax=30,
             cmap='twilight_shifted',
             sweep=0)

In [ ]:
grid_limits = ((0., 15000.), (-50_000., 50_000.), (-50_000., 50_000.))
grid_shape = (31, 201, 201)
uah_grid = pyart.map.grid_from_radars([radar], grid_limits=grid_limits,
                                 grid_shape=grid_shape)
uah_ds = uah_grid.to_xarray()
uah_ds

In [ ]:
uah_ds.isel(z=1).reflectivity.plot(x='lon',
                                   y='lat',
                                   vmin=-20,
                                   vmax=70,
                                   cmap='pyart_NWSRef')

### NEXRAD Data

In [ ]:
import fsspec, s3fs
fs = fsspec.filesystem("s3", anon=True)

In [ ]:
# files = sorted(fs.glob("s3://noaa-nexrad-level2/2008/04/11/KHTX/KHTX20080411_18*"))
# files
files = glob.glob('../../data/nextrad/*')
radar = pyart.io.read(files[1])

Read a single file, the one closes to the UAH volume scan used before

In [ ]:
nexrad_radar = pyart.io.read_nexrad_archive(f's3://{files[4]}', station='KHTX')

In [ ]:
display = pyart.graph.RadarDisplay(nexrad_radar)
display.plot('reflectivity',
                     vmin=-20,
                     vmax=70,
                     cmap='pyart_ChaseSpectral',
                     sweep=0)
plt.ylim(-200, 200)
plt.xlim(-200, 200)

In [ ]:
# Use the ARMOR radar lat/lon as the center for the grid
grid_lat = radar.latitude['data'][0]
grid_lon = radar.longitude['data'][0]

# Make sure the velocity is called corrected velocity to match the corrected data from ARMOR
nexrad_radar.add_field_like('velocity',
                            'corrected_velocity',
                            nexrad_radar.fields['velocity']['data'],
                            replace_existing=True)

nexrad_grid = pyart.map.grid_from_radars([nexrad_radar],
                                         grid_limits=grid_limits,
                                         grid_shape=grid_shape,
                                         grid_origin = (grid_lat, grid_lon),
                                        )

# Convert to xarray and remove the time dimension
nexrad_ds = nexrad_grid.to_xarray().squeeze()

In [ ]:
nexrad_ds.reflectivity.isel(z=1).plot(x='lon',
                                      y='lat',
                                      cmap='Spectral_r',
                                      vmin=-20,
                                      vmax=70)

In [ ]:
nexrad_ds.velocity.isel(z=0).plot(x='lon',
                                  y='lat',
                                  cmap='twilight_shifted',
                                  vmin=-30,
                                  vmax=30)

In [ ]:
u_init, v_init, w_init = pydda.initialization.make_constant_wind_field(
    uah_grid, (0.0, 0.0, 0.0), vel_field='corrected_velocity')

In [ ]:
grids = pydda.retrieval.get_dd_wind_field([uah_grid, nexrad_grid], u_init,
                                          v_init, w_init, Co=1.0, Cm=256.0,
                                          Cx=1e-3, Cy=1e-3, Cz=1e-3, Cb=0.,
                                          frz=5000.0, filter_window=5,
                                          mask_outside_opt=True, upper_bc=1,
                                          vel_name='corrected_velocity',
                                          wind_tol=0.5, engine="tensorflow")

In [ ]:
ds = grids[1].to_xarray()

In [ ]:
ds.reflectivity.isel(z=0, time=0).plot()

In [ ]:
ds.w.isel(z=0, time=0,).plot( x='lon', y='lat')